In [2]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torchvision.datasets import MNIST
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torch.optim import Adam

In [28]:
??MNIST

In [3]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --port 1124

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 1124 (pid 64883), started 5:00:58 ago. (Use '!kill 64883' to kill it.)

In [37]:
class Test(pl.LightningModule):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = nn.Linear(28*28, 128)
        self.layer2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 10)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.softmax(self.layer3(x))
        return x
    def prepare_data(self):
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.1307, ), (0.3081, ))])
                                        
        self.train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        self.test_set = MNIST(os.getcwd(), train=False, transform=transform)
        print('this function has been called')
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=64)
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        return {'val_loss': loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}
    
    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=64)

In [38]:
net = MNISTModel()

In [39]:
trainer = pl.Trainer()
trainer.fit(net)

TypeError: 'MNIST' object is not callable

In [18]:
torch.cuda.available()

AttributeError: module 'torch.cuda' has no attribute 'available'

Dataset MNIST
    Number of datapoints: 10000
    Root location: /home/karmanya/Paper-Challenge
    Split: Test